# OXゲーム

## 問題定義

In [16]:
# Use English for comments
from dataclasses import dataclass
from typing import Iterable, Optional
from game_analyzer import HashState, State, Game, Solver


@dataclass
class OXState(HashState):
    # 0 = empty, 1 = O, 2 = X
    board: list[list[int]]
    current_player: int


class OXGame(Game):
    def __init__(self, board_size: int, line_length: int):
        board = [[0]*board_size for _ in range(board_size)]
        self.board_size = board_size
        self.line_length = line_length
        self.init_state = OXState(board=board, current_player=1)

    def find_next_states(self, state: OXState) -> Iterable[OXState]:
        size = self.board_size
        current_player = state.current_player
        next_player = (state.current_player%2)+1
        state.current_player = next_player
        for r in range(size):
            for c in range(size):
                if state.board[r][c] == 0:
                    state.board[r][c] = current_player
                    yield state
                    state.board[r][c] = 0
        state.current_player = current_player

    def find_mirror_states(self, state: OXState) -> Iterable[OXState]:
        size = self.board_size

        def rotate_90(bd: list[list[int]]) -> list[list[int]]:
            # b'[r][c] = b[size-1-c][r]
            rotated = [[0]*size for _ in range(size)]
            for r in range(size):
                for c in range(size):
                    rotated[r][c] = bd[size - 1 - c][r]
            return rotated

        def flip_horizontal(bd: list[list[int]]) -> list[list[int]]:
            # flip each row horizontally
            flipped = [row[::-1] for row in bd]
            return flipped

        current = state
        for _ in range(4):
            current = rotate_90(current)
            yield current

        current = flip_horizontal(current)

        # Rotate the flipped versions
        for _ in range(4):
            current = rotate_90(current)
            yield current

    def evaluate_state(self, state: OXState) -> int | None:
        # For 3x3 board: 
        # return 1 if X(=2) wins, -1 if O(=1) wins, 0 if draw, else None
        bd = state.board
        size = self.board_size

        # Check rows
        for r in range(size):
            if bd[r][0] == bd[r][1] == bd[r][2] != 0:
                return 1 if bd[r][0] == 2 else -1

        # Check columns
        for c in range(size):
            if bd[0][c] == bd[1][c] == bd[2][c] != 0:
                return 1 if bd[0][c] == 2 else -1

        # Check diagonals
        if bd[0][0] == bd[1][1] == bd[2][2] != 0:
            return 1 if bd[0][0] == 2 else -1
        if bd[0][2] == bd[1][1] == bd[2][0] != 0:
            return 1 if bd[0][2] == 2 else -1

        # Check draw
        # If no empty cell (0) found => draw
        if all(bd[r][c] != 0 for r in range(size) for c in range(size)):
            return 0

        # Not finished yet
        return None


In [18]:
oxgame = OXGame()
solver = Solver()
result = solver.solve(oxgame)

In [19]:
result.state_to_params(oxgame.init_state)

(0, 9)

In [20]:
f"{result.sgg_time=}, {result.ra_time=}"

'result.sgg_time=0.06373977661132812, result.ra_time=0.0009500980377197266'